In [1]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import Select
from selenium.webdriver.common.action_chains import ActionChains
import re, json ,pprint
from bs4 import BeautifulSoup
import pandas as pd
import urllib
import urllib.request
from selenium.webdriver.common.keys import Keys
from IPython.display import Image
from urllib.parse import urlparse
from time import sleep
import datetime
import pytz # new import
new_timezone = pytz.timezone("US/Eastern")
import csv

from dateparser.search import search_dates
import arrow
import datetime
from dateutil.parser import parse
import weasyprint 

/home/mirroraidev/anaconda3/lib/python3.7/site-packages/weasyprint/document.py:34: UserWarning: There are known rendering problems and missing features with cairo < 1.15.4. WeasyPrint may work with older versions, but please read the note about the needed cairo version on the "Install" page of the documentation before reporting bugs. http://weasyprint.readthedocs.io/en/latest/install.html
  'There are known rendering problems and missing features with '


In [2]:
driver_path='./chromedriver'
options = webdriver.ChromeOptions()
options.add_argument('headless')   
driver = webdriver.Chrome(driver_path,options=options)
# driver = webdriver.Chrome('./chromedriver')
driver.implicitly_wait(10)

In [3]:
# my_url = 'https://www.youtube.com/watch?v=A1tqTQkC36A'
my_url = 'https://www.barchart.com/futures/quotes/ZW*0/options?futuresOptionsView=split'
driver.get(my_url)
sleep(1)
# wa
# driver.close()

In [4]:
def load_site(driver_path='./chromedriver',input_url= 'https://www.barchart.com/futures/quotes/ZW*0/options?futuresOptionsView=split'):
    options = webdriver.ChromeOptions()
    options.add_argument('headless')   
    driver = webdriver.Chrome(driver_path,options=options)
    driver.implicitly_wait(10)    
    driver.get(input_url)
    sleep(1)
    return driver

## Save Screenshot

In [5]:
page = driver.page_source
weasyprint.HTML(file_obj=page).write_pdf('out.pdf')

In [6]:
def screenshot_page(webdriver,output_file='out.pdf'):
    page = webdriver.page_source
    weasyprint.HTML(file_obj=page).write_pdf(output_file)

## Create Datestamp

In [37]:
def get_capture_date():
    return datetime.datetime.now().strftime("%a %b %d, %Y")
def get_capture_time(timezone=pytz.UTC):
    return datetime.datetime.now(timezone).strftime("%H:%M:%S %Z")
def get_capture_time_filename(timezone=pytz.UTC):
    return datetime.datetime.now(pytz.UTC).strftime("%Y-%m-%d_%H-%M-%S-%Z")

In [8]:
get_capture_time()

'19:14:07 UTC'

## Extract Name

In [9]:
stock_name = driver.find_element_by_class_name(name='symbol')

In [10]:
stock_name.text

"Wheat Mar '19"

## Extract Days to Expiration

In [11]:
def extract_days_to_expiration(webdriver):
    dt_expiration = webdriver.find_element_by_class_name(name='bc-options-toolbar__second-row')
    a = dt_expiration.find_element_by_xpath("//strong[contains(text(), 'Days')]")
    
    out = re.match(string=a.text,pattern='^([0-9]+)')
    return out.group(1)

In [12]:
dt_expiration = driver.find_element_by_class_name(name='bc-options-toolbar__second-row')
a = dt_expiration.find_element_by_xpath("//strong[contains(text(), 'Days')]")
a.text

'52 Days'

## Extract IV

In [13]:
iv = driver.find_element_by_class_name(name='bc-options-toolbar__second-row')
a = iv.find_element_by_xpath("//div[contains(text(), 'Implied Volatility')]/strong")
a.text

'18.64%'

In [14]:
def extract_iv(webdriver):
    iv = webdriver.find_element_by_class_name(name='bc-options-toolbar__second-row')
    a = iv.find_element_by_xpath("//div[contains(text(), 'Implied Volatility')]/strong")
    return a.text

## Extract Last

In [15]:
iv = driver.find_element_by_class_name(name='pricechangerow')
iv.find_element_by_xpath("//span[@class='last-change']").text

'503-2s'

In [16]:
def extract_last(webdriver):
    iv = webdriver.find_element_by_class_name(name='pricechangerow')
    return iv.find_element_by_xpath("//span[@class='last-change']").text

### Extract Table

In [17]:
test = driver.find_elements_by_xpath("//td[@class='lastPrice_left']")


In [18]:
for item in test:
    print(item.text)

48-6s
44-1s
39-4s
35-2s
31-1s
27-3s
23-7s
20-7s
18-1s
15-5s
13-4s
11-5s
10-0s
8-6s
7-4s
6-4s
5-5s
4-7s
4-2s
3-5s
Last


## Extract Strike

In [19]:
test = driver.find_elements_by_xpath("//td[@class='strikePrice']")
for item in test:
    print(item.text)

455-0
460-0
465-0
470-0
475-0
480-0
485-0
490-0
495-0
500-0
505-0
510-0
515-0
520-0
525-0
530-0
535-0
540-0
545-0
550-0
Strike


## Extract Price Right

In [20]:
test = driver.find_elements_by_xpath("//td[@class='lastPrice_right']")
for item in test:
    print(item.text)

0-5s
0-7s
1-3s
2-1s
3-0s
4-1s
5-6s
7-5s
9-7s
12-4s
15-2s
18-3s
21-6s
25-3s
29-2s
33-1s
37-2s
41-4s
45-7s
50-2s
Last


## Extract Last and Strike

In [21]:
lp_left = driver.find_elements_by_xpath("//td[@class='lastPrice_left']")
strike_price = driver.find_elements_by_xpath("//td[@class='strikePrice']")
lp_right = driver.find_elements_by_xpath("//td[@class='lastPrice_right']")

for i in range(len(lp_left)):
    print(lp_left[i].text,strike_price[i].text,lp_right[i].text)


48-6s 455-0 0-5s
44-1s 460-0 0-7s
39-4s 465-0 1-3s
35-2s 470-0 2-1s
31-1s 475-0 3-0s
27-3s 480-0 4-1s
23-7s 485-0 5-6s
20-7s 490-0 7-5s
18-1s 495-0 9-7s
15-5s 500-0 12-4s
13-4s 505-0 15-2s
11-5s 510-0 18-3s
10-0s 515-0 21-6s
8-6s 520-0 25-3s
7-4s 525-0 29-2s
6-4s 530-0 33-1s
5-5s 535-0 37-2s
4-7s 540-0 41-4s
4-2s 545-0 45-7s
3-5s 550-0 50-2s
Last Strike Last


In [22]:
def extract_left_strike(webdriver):
    lp_left = webdriver.find_elements_by_xpath("//td[@class='lastPrice_left']")
    strike_price = webdriver.find_elements_by_xpath("//td[@class='strikePrice']")
    lp_right = webdriver.find_elements_by_xpath("//td[@class='lastPrice_right']")
    out_list = []
    for i in range(len(lp_left)):
        out_list.append([lp_left[i].text,strike_price[i].text,lp_right[i].text])
    return out_list

In [23]:
driver = load_site()
test_out = extract_left_strike(webdriver=driver)
with open('temp_0.csv', 'w') as csvfile:
    writer = csv.writer(csvfile,delimiter=',',quotechar='"',quoting=csv.QUOTE_NONNUMERIC)
    writer.writerow(["Date",get_capture_date()])
    for i in test_out:
        if(i[0]=='Last'):
            continue
        writer.writerow(i)


In [204]:
with open('temp_0.csv', 'w') as csvfile:
    writer = csv.writer(csvfile,delimiter=',',quotechar='"',quoting=csv.QUOTE_NONNUMERIC)
    for i in range(len(lp_left)):
        if(lp_left[i].text=='Last'):
            continue
        writer.writerow([lp_left[i].text,strike_price[i].text,lp_right[i].text])
        

    #     for row in test_out:
#         writer.writerow(row)

In [24]:
def extract_barchart_site(input_url='https://www.barchart.com/futures/quotes/ZW*0/options?futuresOptionsView=split'\
                         ,name='wheat',write_option='a'):
    driver = load_site(input_url=input_url)
    output_file = name + ".csv"    
    test_out = extract_left_strike(webdriver=driver)
    with open(output_file, write_option) as csvfile:
        writer = csv.writer(csvfile,delimiter=',',quotechar='"',quoting=csv.QUOTE_NONNUMERIC)
        writer.writerow(["Date",get_capture_date()])
        writer.writerow(["Last",extract_last(webdriver=driver)])
        writer.writerow(["Days to expiration",extract_days_to_expiration(webdriver=driver)])
        writer.writerow(["IV",extract_iv(webdriver=driver)])
        writer.writerow(["Time",get_capture_time()])
        writer.writerow([])
        writer.writerow(["Last","Strike","Last"])
        for i in test_out:
            if(i[0]=='Last'):
                continue
            writer.writerow(i)
            
        writer.writerow([])


In [25]:
# extract_barchart_site(write_option='w')
extract_barchart_site(write_option='a')

In [27]:
screenshot_page(webdriver=driver,output_file='wheat.pdf')

In [28]:
get_capture_date()

'Tue Jan 01, 2019'

In [29]:
get_capture_time()

'19:28:58 UTC'

'2019-01-01_19-31-37-UTC'